# JAX on IPU: MNIST neural net training

MNIST neural net training on IPU, following the original JAX tutorial: https://jax.readthedocs.io/en/latest/notebooks/Neural_Network_and_Data_Loading.html

The original example will run as-is on the IPU. Nevertheless, to make full advantage of the IPU hardware and improve user experience, the following modifications were made:

* `jax_platforms = "cpu,ipu"`: use CPU as the default platform for initialization of parameters and dataset.
* `donate_argnums = (0,)` in `update`: buffer donation to keep model parameters on the IPU SRAM between iterations.
* `drop_last = True` in Pytorch dataloader: keep all micro-batches with the same static shape.

In [ ]:
# Install experimental JAX for IPUs
import sys
!{sys.executable} -m pip uninstall -q -y jax jaxlib
!{sys.executable} -m pip install -q https://github.com/graphcore-research/jax-experimental/releases/latest/download/jaxlib-0.3.15-cp38-none-manylinux2014_x86_64.whl
!{sys.executable} -m pip install -q https://github.com/graphcore-research/jax-experimental/releases/latest/download/jax-0.3.16-py3-none-any.whl
!{sys.executable} -m pip install -q torchvision==0.14.0+cpu -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
from jax.config import config

# Use CPU as default backend for initialization.
config.FLAGS.jax_platforms = "cpu,ipu"
# Uncomment to use IPU model emulator.
# config.FLAGS.jax_ipu_use_model = True
# config.FLAGS.jax_ipu_model_num_tiles = 256

In [ ]:
import jax
print("IPU devices:", jax.devices("ipu"))

In [ ]:
import jax.numpy as jnp
from jax import grad, vmap
from jax import random

In [ ]:
# A helper function to randomly initialize weights and biases
# for a dense neural network layer
def random_layer_params(m, n, key, scale=1e-2):
  w_key, b_key = random.split(key)
  return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n,))

# Initialize all layers for a fully-connected neural network with sizes "sizes"
def init_network_params(sizes, key):
  keys = random.split(key, len(sizes))
  return [random_layer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

layer_sizes = [784, 512, 512, 10]
step_size = 0.01
num_epochs = 8
batch_size = 128
n_targets = 10
params = init_network_params(layer_sizes, random.PRNGKey(0))

In [ ]:
from jax.scipy.special import logsumexp

def relu(x):
  return jnp.maximum(0, x)

def predict(params, image):
  # per-example predictions
  activations = image
  for w, b in params[:-1]:
    outputs = jnp.dot(w, activations) + b
    activations = relu(outputs)
  
  final_w, final_b = params[-1]
  logits = jnp.dot(final_w, activations) + final_b
  return logits - logsumexp(logits)

In [ ]:
# This works on single examples
random_flattened_image = random.normal(random.PRNGKey(1), (28 * 28,))
preds = predict(params, random_flattened_image)
print(preds.shape)

In [ ]:
random_flattened_images = random.normal(random.PRNGKey(1), (10, 28 * 28))
# Make a batched version of the `predict` function
batched_predict = vmap(predict, in_axes=(None, 0))
# `batched_predict` has the same call signature as `predict`
batched_preds = batched_predict(params, random_flattened_images)
print(batched_preds.shape)

In [ ]:
from functools import partial

def one_hot(x, k, dtype=jnp.float32):
  """Create a one-hot encoding of x of size k."""
  return jnp.array(x[:, None] == jnp.arange(k), dtype)
  
def accuracy(params, images, targets):
  target_class = jnp.argmax(targets, axis=1)
  predicted_class = jnp.argmax(batched_predict(params, images), axis=1)
  return jnp.mean(predicted_class == target_class)

def loss(params, images, targets):
  preds = batched_predict(params, images)
  return -jnp.mean(preds * targets)

# Explicit jitting for IPU backend.
# Donate `params`` to keep parameters on IPU SRAM. 
@partial(jax.jit, backend="ipu", donate_argnums=(0,))
def update(params, x, y):
  grads = grad(loss)(params, x, y)
  return [(w - step_size * dw, b - step_size * db)
          for (w, b), (dw, db) in zip(params, grads)]

In [ ]:
import numpy as np
from torch.utils import data
from torchvision.datasets import MNIST

def numpy_collate(batch):
  if isinstance(batch[0], np.ndarray):
    return np.stack(batch)
  elif isinstance(batch[0], (tuple,list)):
    transposed = zip(*batch)
    return [numpy_collate(samples) for samples in transposed]
  else:
    return np.array(batch)

class NumpyLoader(data.DataLoader):
  def __init__(self, dataset, batch_size=1,
                shuffle=False, sampler=None,
                batch_sampler=None, num_workers=0,
                pin_memory=False, drop_last=False,
                timeout=0, worker_init_fn=None):
    super(self.__class__, self).__init__(dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        sampler=sampler,
        batch_sampler=batch_sampler,
        num_workers=num_workers,
        collate_fn=numpy_collate,
        pin_memory=pin_memory,
        drop_last=drop_last,
        timeout=timeout,
        worker_init_fn=worker_init_fn)

class FlattenAndCast(object):
  def __call__(self, pic):
    return np.ravel(np.array(pic, dtype=jnp.float32))

In [ ]:
# Define our dataset, using torch datasets
mnist_dataset = MNIST('/tmp/mnist/', download=True, transform=FlattenAndCast())
# IPU: dropping last micro batch with different shape.
training_generator = NumpyLoader(mnist_dataset, batch_size=batch_size, num_workers=0, drop_last=True)

In [ ]:
# Get the full train dataset (for checking accuracy while training)
train_images = np.array(mnist_dataset.train_data).reshape(len(mnist_dataset.train_data), -1)
train_labels = one_hot(np.array(mnist_dataset.train_labels), n_targets)

# Get full test dataset
mnist_dataset_test = MNIST('/tmp/mnist/', download=True, train=False)
test_images = jnp.array(mnist_dataset_test.test_data.numpy().reshape(len(mnist_dataset_test.test_data), -1), dtype=jnp.float32)
test_labels = one_hot(np.array(mnist_dataset_test.test_labels), n_targets)

In [ ]:
import time

params = init_network_params(layer_sizes, random.PRNGKey(0))

for epoch in range(num_epochs):
  start_time = time.time()
  for x, y in training_generator:
    y = one_hot(y, n_targets)
    # `params` update on IPU.
    params = update(params, x, y)
  epoch_time = time.time() - start_time
  
  # Train and test accuracy on CPU.
  params_host = jax.device_get(params)
  train_acc = accuracy(params_host, train_images, train_labels)
  test_acc = accuracy(params_host, test_images, test_labels)
  print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))
  print("Training set accuracy {}".format(train_acc))
  print("Test set accuracy {}".format(test_acc))